In [9]:
import json

with open("tweets.js", "rb") as f:
    tweets = json.loads(f.read().decode("utf-8").replace("window.YTD.tweets.part0 = ", ""))

In [16]:
tweets[108]

{'tweet': {'edit_info': {'initial': {'editTweetIds': ['402454829244493824'],
    'editableUntil': '2013-11-18T15:44:43.198Z',
    'editsRemaining': '5',
    'isEditEligible': True}},
  'retweeted': False,
  'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
  'entities': {'hashtags': [{'text': 'OSDFCon', 'indices': ['45', '53']}],
   'symbols': [],
   'user_mentions': [{'name': 'BasisTech',
     'screen_name': 'basistechnology',
     'indices': ['6', '22'],
     'id_str': '104599119',
     'id': '104599119'}],
   'urls': [{'url': 'http://t.co/U7626e54gg',
     'expanded_url': 'http://cdn2.hubspot.net/hub/165417/file-377400132-pdf/presentations/osdf-2013/Willi_Ballenthin.pdf',
     'display_url': 'cdn2.hubspot.net/hub/165417/fil…',
     'indices': ['106', '128']}]},
  'display_text_range': ['0', '134'],
  'favorite_count': '7',
  'id_str': '402454829244493824',
  'truncated': False,
  'retweet_count': '4',
  'id': '402454829244493824',
  'possibly_sensitive'

In [11]:
def is_retweet(tweet):
    # this seems to be the format rendered for retweets.
    # surprisingly there doesn't seem to be a flag in the metadata?
    return tweet["tweet"].get("full_text").startswith("RT @")

In [13]:
def is_reply(tweet):
    return tweet["tweet"].get("in_reply_to_screen_name") or tweet["tweet"].get("full_text", "").startswith("@")

In [15]:
import io
import pathlib
import textwrap
import datetime

for i, tweet in enumerate(tweets):
    if is_retweet(tweet): continue
    if is_reply(tweet): continue

    full_text = tweet["tweet"].get("full_text", "")

    # replace t.co tracker links to the original link
    for url in tweet["tweet"].get("entities", {}).get("urls", []):
        full_text = full_text.replace(url["url"], url["expanded_url"])

    # replace t.co image links to the twimg link
    images = []
    for media in tweet["tweet"].get("entities", {}).get("media", []):
        if media.get("type") != "photo": continue

        # like: "FdLWzPnXoAAR4v0.jpg"
        basename = media["media_url"].rpartition("/")[2]

        # filename provided by twitter export,
        # found in directory tweet_media/
        # like: "1572555484137426945-FdLWzPnXoAAR4v0.jpg"
        imgname = tweet["tweet"]["id_str"] + "-" + basename

        images.append(imgname)

        # strip inline image link
        full_text = full_text.replace(media["url"], "")

    tags = []
    for hashtag in tweet["tweet"].get("entities", {}).get("hashtags", []):
        tags.append(hashtag["text"])

    title = "tweet: " + full_text.partition("\n")[0][:32]

    print(i, full_text, images or "")

    if i not in (7, 16, 19, 108):
        continue

    id = tweet["tweet"]["id_str"]
    date = datetime.datetime.strptime(tweet["tweet"]["created_at"], "%a %b %d %H:%M:%S %z %Y")

    post = io.StringIO()
    post.write("---\n")
    post.write(f'title: "{title}"\n')
    post.write(f'slug: "{id}"\n')
    post.write(f'date: "{date.isoformat()}"\n')
    if tags:
        post.write('tags:\n')
        for tag in tags:
            post.write(f'  - "{tag}"\n')
    if images:
        post.write('images:\n')
        for image in images:
            post.write(f'  - "{image}"\n')
    post.write("---\n")
    post.write("")
    post.write(full_text)
    post.write("")

    with open(f"{id}.md", "wb") as f:
        f.write(post.getvalue().encode("utf-8"))

7 going to try to default to Mastodon instead of the Blue Bird for the next week and see what that’s like.

My (empty) profile is on @mastodon.social 
8 “I have a soft spot for shellcode payloads consisting of plugins that invoke each other for functionality. Way overengineered but conceptually elegant.” 
12 Rust in the Linux kernel, Rust in Windows DirectWrite, … 
16 there’s a WMI/CIM database parser, cool!

https://github.com/fox-it/dissect.cim https://twitter.com/_dirkjan/status/1577303500706439170 
19 This truck is like a giant dentist’s Waterpik: it pushes water through one hose and then sucks it back through another hose to clean confined spaces.  ['1572555484137426945-FdLWzPnXoAAR4v0.jpg']
21 This is a grape harvesting vehicle. I’d estimate it covers 50m of vines every minute. It’s a really neat design!  ['1571884636841582594-FdB0VdRXgAM7Qgl.jpg']
23 capa now supports .NET programs! https://twitter.com/m_r_tz/status/1557428559995703302 
27 free idea: a FUSE filesystem that gener